In [2]:
import cv2
import numpy as np
from PIL import Image
import os
import random

In [3]:
synthetic_face_dir = '../face_generation/generated_faces'  # Adjusted path
synthetic_faces = []
try:
    for filename in os.listdir(synthetic_face_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(synthetic_face_dir, filename)
            img = Image.open(img_path).convert("RGB")
            synthetic_faces.append(np.array(img))
    print(f"Loaded {len(synthetic_faces)} synthetic faces.")
    if not synthetic_faces:
        print("Warning: No synthetic faces found in the directory.")
except FileNotFoundError:
    print(f"Error: Directory '{synthetic_face_dir}' not found.")
    exit()



Loaded 5 synthetic faces.


In [5]:
video_path = 'input_video.mp4'  # Corrected path: video in the same directory
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file: {video_path}")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('anonymized_video.mp4', fourcc, fps, (frame_width, frame_height)) # Corrected path: output in the same directory


In [8]:
face_cascade_path = 'haarcascade_frontalface_default.xml' # Corrected path: Haar cascade in the same directory
face_cascade = cv2.CascadeClassifier(face_cascade_path)
if face_cascade.empty():
    print("Error: Haar cascade classifier not loaded.")
    cap.release()
    output_video.release()
    exit()


In [10]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        if synthetic_faces:
            random_face = random.choice(synthetic_faces)
            resized_face = cv2.resize(random_face, (w, h))
            frame[y:y+h, x:x+w] = resized_face

    output_video.write(frame)

In [12]:
cap.release()
output_video.release()
print("Anonymization complete. Output video saved as anonymized_video.mp4")

Anonymization complete. Output video saved as anonymized_video.mp4
